**Question 1**.
**Exploring a Real-World Dataset**

Let's analyze a dataset with features that include whether it's a weekday, weather conditions (such as temperature and humidity), and more. The target variable is the count of bike rentals. If you are curious about this dataset, you can read more at [UCI Machine Learning Repository - Bike Sharing Dataset](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset). However, detailed descriptions of the dataset are not necessary to complete this exercise. Our goal is to predict the number of bike rentals.

1. What do you think is a reasonable distribution to model the target variable?

* A: Poisson Distribution since we are trying to count the number of bike rentals within an amount of time.

2. By MLE, derive the cost function and write the derivation in latex.

* A:
$$
f(y) = \frac{\lambda^{y}e^{-\lambda}}{y!} \\

L(\mu) = \Pi_{i=1}^{n}{\frac{\lambda^{y_i}e^{-\lambda}}{y_i!}} \\

l(\mu) = log_{10}(\mu) = \sum_{i=1}^{n}log{\frac{\lambda^{y_i}e^{-\lambda}}{y_i!}} = \sum_{i=1}^{n}({y_i{log{\lambda}} - \lambda - log{y_i!}}) = \sum_{i=1}^{n}({y_i{log{\lambda}} - \lambda})
$$

The x_i factorial term above is dropped as it is independent of lambda.

Additionally, we will minimise the cost function instead of maximising the likelihood function. Hence,

$$
l(\mu) = -\sum_{i=1}^{n}({y_i{log{\lambda}} - \lambda}) \\

\lambda_i = e^{x_i^{T}w}
$$

Therefore,

$$
l(\mu) = -\sum_{i=1}^{n}({y_i{log{e^{x_i^{T}w}}} - e^{x_i^{T}w}}) = -\sum_{i=1}^{n}({y_i{x_i^{T}w} - e^{x_i^{T}w}})
$$

Scaling it down by 1/n,

$$
\frac{l(\mu)}{n} = -\frac{1}{n}\sum_{i=1}^{n}({y_i{x_i^{T}w} - e^{x_i^{T}w}})
$$
$$

3. Compute the gradient of this objective function $\min_w -\frac{1}{n} \sum_{i=1}^n (y_i x_i^\top w - e^{x_i^\top w})$.


Hint: try to compute the gradient of a single data point, then you can sum over all data and scaled it by $1/n$. The summation can be done without using for loop. As you already learned, for loop can be very slow.

* A:

$$
\nabla{\frac{l(\mu)}{n}} = \frac{d}{dw}{(-\frac{1}{n}\sum_{i=1}^{n}({y_i{x_i^{T}w} - e^{x_i^{T}w}}))} = -\frac{1}{n} \frac{d}{dw}{\sum_{i=1}^{n}({y_i{x_i^{T}w} - e^{x_i^{T}w}})} = -\frac{1}{n}{\sum_{i=1}^{n}{(y_i{x_i}-e^{x_i^{T}{w}}x_i)}} = -\frac{1}{n}{\sum_{i=1}^{n}{x_i{(y_i-e^{x_i^{T}w})}}}
$$

4. Optimizing the above objective function by gradient descent. The goal is to make reasonable predictions of bike rentals. Part of the code is already given.

In [2]:
# The original dataset is large, below I provide a cleaned subset.
# NO need to edit this block
import numpy as np
import pandas as pd

np.random.seed(0)
fileloc = 'https://raw.githubusercontent.com/yannickycpan/oxford-engs-AIML-cwm/main/subsetbikeshare.txt'
dataset = pd.read_csv(fileloc)
dataset = dataset.values
np.random.shuffle(dataset)
print(dataset.shape)
np.random.shuffle(dataset)
# I already put the target variable the last column
Y = dataset[:, -1]
X = dataset[:, :-1]
X = np.hstack((X, np.ones((X.shape[0], 1))))

valn = int(0.1*X.shape[0])
Yval = Y[:valn]
Xval = X[:valn]

X=X[valn:]
Y=Y[valn:]

print(Y[:10])

def rmse(ytrue, ypred):
  return np.sqrt(np.mean((ytrue-ypred)**2))

(4999, 115)
[ 50.   5. 225.  28. 102. 313.   4. 332. 576.  92.]


In [3]:
print(X.shape, Y.shape)

(4500, 115) (4500,)


In [4]:
def grad(w):
  # TODO: complete the code here to compute gradient
  yhat = np.exp(X@w)
  return -X.T@(Y-yhat)/len(Y)

lr = 0.001  # learning rate

w = np.zeros(X.shape[1])
for i in range(3000):
  # TODO: complete the gradient updating rule
  w -= lr * grad(w)

  if i%100==0: # print out the root mean squared error every 100 steps
    # TODO: please replace the question marks below
    Yvalpred = np.exp(Xval@w)
    Ypred = np.exp(X@w)
    print(' validation err is :: ', rmse(Yval, Yvalpred))
    print(' train err is :: ', rmse(Y, Ypred))

 validation err is ::  253.05831260355257
 train err is ::  258.14321143327703
 validation err is ::  101.0602113436416
 train err is ::  98.45896719849128
 validation err is ::  94.10932692441442
 train err is ::  90.89283554975972
 validation err is ::  91.5415732336252
 train err is ::  88.30430024731653
 validation err is ::  90.37403498987621
 train err is ::  87.20876925256977
 validation err is ::  89.77517802445003
 train err is ::  86.65421511784442
 validation err is ::  89.45198825258652
 train err is ::  86.33751629982505
 validation err is ::  89.27361891563514
 train err is ::  86.1400895073618
 validation err is ::  89.17457770714961
 train err is ::  86.00837906547643
 validation err is ::  89.12030621738121
 train err is ::  85.91556327300144
 validation err is ::  89.09192669761997
 train err is ::  85.84706964988898
 validation err is ::  89.07882076176318
 train err is ::  85.79446011443952
 validation err is ::  89.07485451818228
 train err is ::  85.75260409752687

5. Test the linear regression model we learned previously on this dataset. How does it compare with the algorithm above? You should find that by incorporating appropriate prior knowledge—specifically (i.e., recognizing that the target variable represents counts—into the model building process), we can achieve better performance.

* A: The smaller validation and training error shows that the result improves after adding prior knowledge.

Linear regression:

$$
L(w) = \frac{1}{2n} \sum_{i=1}^{n} (y_i - x_i^\top w)^2 \\

\nabla_w L(w) = -\frac{1}{n} \sum_{i=1}^{n} (y_i - x_i^\top w) x_i
$$

In [5]:
# TODO: compute the gradient of the linear regression objective
def grad(lrw):
  y_hat = X.dot(lrw)
  return -X.T.dot(Y-y_hat)/len(Y)

# feel free to change the learning rate if you want
lrw = np.zeros(X.shape[1])
for i in range(3000):
  lrw = lrw - 0.01 * grad(lrw)

  if i%100==0:
    # TODO: print out the root mean squared error every 100 steps
    # please replace the question marks below
    y_val_pred = Xval.dot(lrw)
    y_train_pred = X.dot(lrw)
    print('validation err is :: ', rmse(Yval, y_val_pred))
    print('train err is :: ', rmse(Y, y_train_pred))

# (optional) TODO: you can also use closed-form solution
# and directly print the validation and training error below:

validation err is ::  247.978675572676
train err is ::  253.1243774289587
validation err is ::  148.1711421747072
train err is ::  155.98531070740694
validation err is ::  139.79645597569143
train err is ::  146.8628697852999
validation err is ::  133.29832790051137
train err is ::  139.65442832386384
validation err is ::  128.14023471220395
train err is ::  133.84976217890463
validation err is ::  123.9960266660276
train err is ::  129.11878096894543
validation err is ::  120.63525217105324
train err is ::  125.22759900111316
validation err is ::  117.88695035325014
train err is ::  122.00219515995211
validation err is ::  115.62074161070257
train err is ::  119.30912222785277
validation err is ::  113.73576652366687
train err is ::  117.0442615934371
validation err is ::  112.15348360963696
train err is ::  115.12560143437135
validation err is ::  110.81256431563992
train err is ::  113.48820367922805
validation err is ::  109.66507215056598
train err is ::  112.08048768927002
valida

**Question 2**. Assume that you know some feature columns (i.e. the column indexes) are highly inaccurate on the above bikeshare dataset. Could you design a new regularization method to help improve the solution? Note that you are NOT required to implement your method, but you should write the math expression of the regularization you designed in an implementable way and provide some nice/reasonable explanations to your design. If you want to do coding test for your new regularization, feel free to add a coding cell below.

We can define a new regularization term that imposes a stronger penalty on the weights corresponding to the inaccurate features. One way to do this is by introducing different regularization strengths for the accurate and inaccurate features.

The overall loss function with this new regularization term can be written as:
$
L(w)=\frac{1}{n}{\sum_{i=1}^{n}{l(y_i,\hat y_i)+\lambda R(w)}}
$

where, l is the loss function and lambda is the regularisation parameter.

By imposing a stronger penalty on the inaccurate features, we effectively reduce their impact on the model.
Additionally, the method allows for different levels of regularization, which can be tuned based on the degree of inaccuracy known for each feature. This flexibility can be useful in practice where some features are more inaccurate than others.

**Question 3 Two subquestions about the derivation of the cost function for multiclass classification problems**. `

Modeling multiclass data. Logistic regression we learned above is good for binary class. What if we have $m>2$ classes? We will assume multinomial distribution. You will finally get the very popular softmax cross entropy loss. In the rest of this question, a vector is a column vector unless otherwise specified.

To deal with multiclass classification problem, it is common to assume that $p(y|x)$ is multinomial distribution (some places call categorical distribution as it is a special case of multinomial distribution), and apply MLE. Given a sample $x, y$, we can write (a slight notation abuse: $y_i \in \{0,1\}$ here means the ith component in the one-hot vector y which is m-dimensional). The conditional distribution of a data point $x, y$ is:
\begin{align}
p(y|x) &= \frac{1}{y_1 ! ... y_m !} p(y_1=1|x)^{y_1}...p(y_m=1|x)^{y_m}
\end{align}
We can parameterize those $m$ probabilities by a softmax function: $\sigma(x^\top W), W \in \mathbb{R}^{d\times m}$. The softmax function is defined as: $\sigma :\mathbb {R} ^{m}\to (0,1)^{m}, $
$\sigma (z)_{i}={\frac {e^{z_{i}}}{\sum _{j=1}^{m}e^{z_{j}}}}\ \ {\text{ for }}i=1,\dotsc ,m{\text{ and }} z =(z_{1},\dotsc ,z_{m})\in \mathbb {R} ^{m}.$

Note:
- one-hot vector is the one with a single entry to be one and other $m-1$ entries are zero. It is common to convert an integer label to one-hot vector in classification problems. For integer $i$, the ith entry is 1 otherwise zero. For example, for integer zero, the one-hot vector is $(1, 0, ..., 0)$. Here, we have m classes (m possible labels from 0, 1, 2, ..., m-1). As a result, the one-hot vector is m-dimensional.

- in this question we use superscript to denote a sample, subscript to denote a component in a vector.

- $x^{(i)}$ is a column vector $d-by-1$.

1. Based on the above hints, please complete the derivation to get the cost function below.

Finally, we arrive the following cost function:
$
\min_w \frac{1}{n} \sum_{i=1}^n \log(\mathbf{1}^\top \exp{(x^{(i) \top} W)}^\top ) - x^{(i)\top} W y^{(i)}
$  
where I use superscripe to denote the $i$th training example and $\mathbf{1}$ is a $m$-dimensional column vector with all ones.

* A:

Likelihood of y given x:

\begin{align*}
\
p(y|x) = \prod_{j=1}^m \left( \frac{\exp(x^\top W_j)}{\sum_{k=1}^m \exp(x^\top W_k)} \right)^{y_j}
\
\end{align*}

Taking its log:

\begin{align*}
\
\log p(y|x) = \log \left( \prod_{j=1}^m \left( \frac{\exp(x^\top W_j)}{\sum_{k=1}^m \exp(x^\top W_k)} \right)^{y_j} \right) = \sum_{j=1}^m y_j \log \left( \frac{\exp(x^\top W_j)}{\sum_{k=1}^m \exp(x^\top W_k)} \right) = \sum_{j=1}^m y_j \left( x^\top W_j - \log \left( \sum_{k=1}^m \exp(x^\top W_k) \right) \right)
\
\end{align*}

Dividing by n and converting element-wise to vector/matrix, cost function:

\begin{align*}
\
\\min_W \frac{1}{n} \sum_{i=1}^n \left( \log \left( \mathbf{1}^\top \exp \left( x^{(i) \top} W \right) \right) - x^{(i) \top} W y^{(i)} \right)
\
\end{align*}

2. **[optional]** You may also see a cost function for multiclassification looks like $-\frac{1}{n} \sum_{i=1}^n y^{(i)\top} \log(softmax(x^{(i) \top} W)^\top)$. Is it equivalent to the above cost function? Please prove your answer.

**Question 4 [optional]**

Based on your understanding of MLE, develop a binary classifier that models the positive class as:

$
p(y=1|x,w)=\frac{1}{2}(1 + \frac{w^\top x}{\sqrt{1+(w^\top x)^2}})
$

You need to write the log-likelihood function and compute the gradient.